This notebook utilizes the Workbench and Client classes, which allows for convenient, notebook-based use of the sdk.

In [ ]:
from datetime import time, timedelta
from whendo.sdk.client import Client
from whendo.sdk.workbench import Workbench
import whendo.core.util as util
import whendo.core.actions.file_action as file_x
import whendo.core.actions.logic_action as logic_x
import whendo.core.scheduler as sched_x
import whendo.core.actions.gpio_action as gpio_x
import whendo.core.actions.sys_action as sys_x

"""
define the actions
"""
green_on = gpio_x.SetPin(pin=27, on=True)
green_off = gpio_x.SetPin(pin=27, on=False)
green_toggle = gpio_x.TogglePin(pin=27)
red_on = gpio_x.SetPin(pin=25, on=True)
red_off = gpio_x.SetPin(pin=25, on=False)
red_toggle = gpio_x.TogglePin(pin=25)
gpio_clear = gpio_x.Cleanup()
toggle_toggle = logic_x.All(action_list=[green_toggle, red_toggle])
on_on = logic_x.All(action_list=[green_on, red_on])
off_off = logic_x.All(action_list=[green_off, red_off])
file_heartbeat = file_x.FileHeartbeat(file="gpio_beat.txt")
system_info = sys_x.SysInfo()
pause1 = sys_x.Pause()
pause2 = sys_x.Pause(seconds=2)
start_pivot = logic_x.All(action_list=[red_on, green_on, pause1, green_off])

"""
define the schedulers
"""
often = sched_x.Timely(interval=1)
morning, evening = time(6,0,0), time(18,0,0)
daily_often = sched_x.Timely(start=morning, stop=evening, interval=1)
nightly_often = sched_x.Timely(start=evening, stop=morning, interval=1)
randomly_often = sched_x.Randomly(time_unit=util.TimeUnit.second, low=2, high=5)
timely_at_00_sec = sched_x.Timely(interval=1, second=00)
timely_at_30_sec = sched_x.Timely(interval=1, second=30)
timely_at_10_sec = sched_x.Timely(interval=1, second=10)
timely_every_10 = sched_x.Timely(interval=10)
timely_every_30 = sched_x.Timely(interval=30)
immediately = sched_x.Immediately()


def add_actions(client):
    """
    add actions to the 'client' dispatcher
    """
    [ client.add_action(*action) for action in [
        ('green_on', green_on),
        ('green_off', green_off),
        ('green_toggle', green_toggle),
        ('red_on', red_on),
        ('red_off', red_off),
        ('red_toggle', red_toggle),
        ('gpio_clear', gpio_clear),
        ('toggle_toggle', toggle_toggle),
        ('on_on', on_on),
        ('off_off', off_off),
        ('file_heartbeat', file_heartbeat),
        ('sys_info', system_info),
        ('pause1', pause1),
        ('pause2', pause2),
        ('start_pivot', start_pivot)
        ]
    ]

def add_schedulers(client):
    """
    add schedulers to the 'client' dispatcher
    """
    [ client.add_scheduler(*scheduler) for scheduler in [
        ('often', often),
        ('daily_often', daily_often),
        ('nightly_often', nightly_often),
        ('randomly_often', randomly_often),
        ('timely_at_00_sec', timely_at_00_sec),
        ('timely_at_10_sec', timely_at_10_sec),
        ('timely_at_30_sec', timely_at_30_sec),
        ('timely_every_10', timely_every_10),
        ('timely_every_30', timely_every_30),
        ('heartbeat', timely_every_30),
        ('immediately', immediately)
        ]
    ]

def schedule_actions(client):
    """
    schedule the actions of interest
    """
    if True:
        now = util.Now.dt()
        delta_20 = timedelta(seconds=20)
        dt1 = now + timedelta(seconds=10)
        dt2 = now + timedelta(minutes=3)
        dt1_plus = dt1 + delta_20
        dt2_minus = dt2 - delta_20
        client.defer_action("immediately", "start_pivot", util.DateTime(date_time=dt1))
        client.defer_action("heartbeat", "red_on", util.DateTime(date_time=dt1_plus))
        client.expire_action("heartbeat", "red_on", util.DateTime(date_time=dt2_minus))
        client.defer_action("immediately", "gpio_clear", util.DateTime(date_time=dt2))
    elif True:
        now = util.Now.dt()
        delta_20 = timedelta(seconds=20)
        dt_20 = now + delta_20
        dt_40 = dt_20 + delta_20
        dt_60 = dt_40 + delta_20
        client.schedule_action("often", "toggle_toggle")
        client.expire_action("often", "toggle_toggle", dt_20)
        client.defer_action("often", "on_on", dt_20)
        client.expire_action("often", "on_on", dt_40)
        client.defer_action("often", "toggle_toggle", dt_40)
        client.expire_action("often", "toggle_toggle", dt_60)
    elif True:
        [ client.schedule_action(*stuff) for stuff in [
            ('randomly_often', 'red_toggle'),
            ('daily_often', 'green_toggle'),
            ('nightly_often', 'green_toggle')
            ]
    ]

def unschedule_schedulers(client):
    """
    unschedule the schedulers
    """
    [ client.unschedule_scheduler(scheduler) for scheduler in [ 
        'randomly_often', 'daily_often', 'nightly_often', 'often'
        ]
    ]
def all_on(client):
    client.execute_action('green_on')
    client.execute_action('red_on')

def all_off(client):
    client.execute_action('green_off')
    client.execute_action('red_off')

def sys_info(client):
    print(client)
    util.PP.pprint(client.execute_action('sys_info'))

"""
set up workbench
"""
workbench = Workbench()
workbench.add_client('pi4', Client(host='192.168.0.45'))
workbench.add_client('pi3', Client(host='192.168.0.46'))
local = workbench.get_client('local')
pi3 = workbench.get_client('pi3')
pi4 = workbench.get_client('pi4')

"""
convenience functions applied to a list of Clients
"""
def init(client_list):
    for c in client_list:
        c.clear_dispatcher()
        add_actions(c)
        add_schedulers(c)
        schedule_actions(c)
def run(client_list):
    for c in client_list:
        c.run_jobs()
def stop(client_list):
    for c in client_list:
        c.stop_jobs()
def pprint(client_list):
    for c in client_list:
        print(f"job count={c.job_count()} for ({c})")
        c.load_dispatcher().pprint()
def on(client_list):
    for c in client_list:
        all_on(c)
def off(client_list):
    for c in client_list:
        all_off(c)
def info(client_list):
    for c in client_list:
        sys_info(c)


In [ ]:
pprint([pi3, pi4])

In [ ]:
init([pi3, pi4])

In [ ]:
run([pi3, pi4])

In [ ]:
stop([pi3, pi4])

In [ ]:
on([pi3, pi4])

In [ ]:
off([pi3, pi4])

In [ ]:
info([pi3, pi4])

In [ ]:
pi3.execute_action("gpio_clear")